In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from spark.spark_config import create_spark_session
spark = create_spark_session()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d749102f-b484-4096-be97-5180a8145bcd;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [3]:
from data_loader import DataLoader
from preprocessor import Preprocessor
from nlp_processor import NLPProcessor
from feature_extractor import FeatureExtractor


In [4]:
data_loader = DataLoader(spark)
df = data_loader.load_sample()

In [5]:
# analyser = EdaAnalyser(df)
# analyser.run_full_eda_report(['title', 'lyrics', 'views'])


In [6]:
df = Preprocessor.run(df)


In [7]:
df_tokenized = NLPProcessor.run(df)
df_tokenized.cache()

DataFrame[title: string, tag: string, artist: string, year: int, views: int, features: string, lyrics: string, id: int, language_cld3: string, language_ft: string, language: string, lyrics_cleaned: string, words_lemmatized: array<string>]

In [8]:
extractor = FeatureExtractor()
# extractor.fit(df_tokenized)
# df_final = extractor.transform(df_tokenized)

In [9]:
from pathlib import Path
base_dir = Path.cwd().parent
test_file_path = base_dir / "models" / "test" / "word2vec"
# extractor.save(str(test_file_path))
df_final = extractor.load(str(test_file_path)).transform(df_tokenized)

In [10]:
df_final.select("feature_array").show(1)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `feature_array` cannot be resolved. Did you mean one of the following? [`features`, `year`, `artist`, `language`, `language_cld3`].;
'Project ['feature_array]
+- Project [title#0, tag#1, artist#2, year#3, views#4, features#5, lyrics#6, id#7, language_cld3#8, language_ft#9, language#10, lyrics_cleaned#34, words_lemmatized#89, UDF(words_lemmatized#89) AS word_vectors#196]
   +- Project [title#0, tag#1, artist#2, year#3, views#4, features#5, lyrics#6, id#7, language_cld3#8, language_ft#9, language#10, lyrics_cleaned#34, words_lemmatized#89]
      +- Project [title#0, tag#1, artist#2, year#3, views#4, features#5, lyrics#6, id#7, language_cld3#8, language_ft#9, language#10, lyrics_cleaned#34, words_tokens#50, words_filtered#70, lemmatize_udf(words_filtered#70)#88 AS words_lemmatized#89]
         +- Project [title#0, tag#1, artist#2, year#3, views#4, features#5, lyrics#6, id#7, language_cld3#8, la

In [11]:
from repository.cassandra_provider import CassandraProvider 
cassandra_provider = CassandraProvider()

In [12]:
cassandra_provider.save(df_final)

26/01/01 21:54:02 WARN TaskSetManager: Lost task 0.0 in stage 7.0 (TID 9) (172.20.0.5 executor 0): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 814, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 650, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 375, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 87, in read_command
    command = serializer._read_with_length(file)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 472

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 814, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 650, in read_udfs
    udfs.append(read_single_udf(pickleSer, infile, eval_type, runner_conf, udf_index=i))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 375, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 87, in read_command
    command = serializer._read_with_length(file)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 472, in loads
    return cloudpickle.loads(ob

In [ ]:
Podsumowanie poprawek
Zmień bind mount na named volume (cassandra_data), aby uniknąć błędów "Permission denied".

Dodaj zmienne środowiskowe CASSANDRA_USER i CASSANDRA_PASSWORD.

Ustaw CASSANDRA_AUTHENTICATOR na PasswordAuthenticator, bo domyślnie Cassandra pozwala na wejście każdemu bez hasła (AllowAllAuthenticator).